## TO TEST NEW CODE

In [ ]:
import os
import skimage
import astropy

# Aliases
import numpy as np
import matplotlib.pyplot as plt

from cartesian_to_polar import CartesianToPolar

In [ ]:
# Import the images

path = os.path.join(os.getcwd(), '..', '..', 'sdo')
filepath = os.path.join(path, 'AIA_fullhead_000.fits.gz')
hdul = astropy.io.fits.open(filepath)

In [ ]:
new_polar = CartesianToPolar.get_polar_image(
    image_nb=1,
    output_shape=(10_000, 10_000),
    borders= {
        'radial distance': (690, 870),
        'polar angle': (245, 295),
    },
    direction='clockwise',
    theta_offset=90,
    channel_axis=None,
)

In [ ]:
plt.figure()
plt.imshow(new_polar.T, interpolation='none', origin='lower')
plt.xlabel('radial distance')
plt.ylabel('polar angle')
plt.colorbar()
plt.show()

In [ ]:
image = hdul[0].data

plt.figure()
plt.imshow(image, interpolation='none')
plt.colorbar()
plt.show()

In [ ]:
header = hdul[0].header
header

In [ ]:
# Image to polar
rsun_image = header
image_polar = skimage.transform.warp_polar(
    image,
    center=(header['X0_MP'], header['Y0_MP']),
    output_shape=(4096, 4096),
    
) #TODO: add radius later

In [ ]:
plt.figure(figsize=(14, 8))
plt.imshow(image_polar, interpolation='none')
# plt.xlim(1600, 1900)
# plt.ylim(140, 220)
plt.show()

In [ ]:
np.min(new_polar)

X0_MP: Master pointing X0 sun center in CCD frame in pixels, start 0.0 (float) for raw image. so the first pixel is labeled as 0?
columns is the x axis. so plt.plot(x, y) should be in the right order I think.

other import keys might be CDELT, RSUN_REF, RSUN_OBS,
Do I need SAT_ROT?

In [ ]:
# Try and get the d_theta for each pixel
def get_d_theta(image: np.ndarray) -> float:

    theta_length = image.shape[1]
    print(f'theta length is {theta_length}')

    d_theta = 360 / theta_length
    print(f'd_theta is {d_theta}')
    return d_theta

In [ ]:
d_theta = get_d_theta(image_polar)

In [ ]:
plt.figure(figsize=(14, 8))
plt.imshow(image_polar.T, interpolation='none', aspect='auto')
# plt.xlim(245 / d_theta, 295 / d_theta)
# plt.ylim(140, 220)
plt.show()

In [ ]:
def cartesian_to_polar(
        image: np.ndarray,
        center: tuple[int | float, int | float],
        direction: str,
    ) -> dict[str, np.ndarray]:
    
    x, y = np.nonzero(image)
    x_0, y_0 = center

    radial_distance = np.sqrt((x - x_0)**2 + (y - y_0)**2)
    polar_angle = np.arctan2(y - y_0, x - x_0)

    polar_coordinates = {
        'radial distance': radial_distance,
        'polar angle': (polar_angle + np.pi) * 180 / np.pi,
    }
    return polar_coordinates



In [ ]:
test_dict = cartesian_to_polar(image, center=(header['X0_MP'], header['Y0_MP']))
plt.figure(figsize=(14, 8))
plt.scatter(test_dict['polar angle'], test_dict['radial distance'], s=0.01)
# plt.imshow(image_polar.T, interpolation='none', aspect='auto')
plt.xlim(0, 360)
plt.ylim(0, 2000)
plt.show()

## IMPORTANT

In [ ]:
class CartesianToPolar:
    """
    To change coordinates, or an image, from the cartesian representation to polar coordinates.
    """

    def __init__(
            self,
            image: np.ndarray | None,
            center: np.ndarray | tuple[int | float, int | float],
            output_shape: tuple[int, int],
            borders: dict[str, any],
            direction: str = 'anticlockwise',
            theta_offset: int | float = 0,
            channel_axis: None | int = None,
            **kwargs,
        ) -> None:

        # Attributes
        self.image = image
        self.center = center
        self.output_shape = output_shape
        self.borders = borders
        self.direction = direction
        self.theta_offset = theta_offset
        self.channel_axis = channel_axis

        self.kwargs = kwargs

        # Setup
        self._initial_checks()
        
    
    def _initial_checks(self) -> None:

        # Direction keyword argument check
        direction_options = ['solar', 'clockwise', 'anticlockwise']
        if self.direction not in direction_options:
            raise ValueError(f"'{self.direction} not in permitted options. You need to choose between {', '.join(direction_options)}.")
        elif self.direction == 'anticlockwise':
            polar_coordinates = self._coordinates_cartesian_to_polar()


    def _option_setup(self):

        if self.direction == 'anticlockwise':
            polar_coordinates = self._coordinates_cartesian_to_polar() #TODO: need to add the theta offset
            #TODO: need to add the option of the values if they exist or maybe do it later if I recreate the final image

        #TODO: this might all be useless, need to check the .warp_polar documentation

    def _coordinates_cartesian_to_polar(self) -> dict[str, np.ndarray]:
        """
        To change the cartesian coordinates to the polar ones

        Returns:
            dict[str, np.ndarray]: the corresponding radial distance and polar angle as np.ndarrays.
        """

        x, y = self.coordinates
        
        # Polar coordinates
        radial_distance = np.sqrt((x - self.center[0])**2 + (y - self.center[1])**2)
        polar_angle = np.arctan2(y - self.center[1], x - self.center[0])

        polar_coordinates = {
            'radial distance': radial_distance,
            'polar angle': (polar_angle + np.pi) * 180 / np.pi,
        }
        return polar_coordinates
        